# Bias Detection with **MSD**

**Maximum Subgroup Discrepancy (MSD)** measures how far two distributions differ inside their worst-case protected subgroup.

* Two datasets are **close** iff every subgroup defined by the
  protected attributes behaves similarly.  
* MSD keeps that worst-case guarantee **and** needs only **O(# protected features)**
  samples (linear) instead of the exponential sample sizes required by
  classical distances such as Total Variation or Wasserstein.

**Why this matters**: MSD uncovers hidden *intersectional* bias that
  marginal metrics miss **and** returns the exact logical rule describing the
  most disadvantaged group.


## Demo dataset  `01_data.csv`

| Column   | Type & values                                                 | Role            |
|----------|--------------------------------------------------------------|-----------------|
| `Race`   | categorical {Green, Blue, Purple}                            | protected       |
| `Age`    | categorical {0-18, 18-30, 30-45, 45-60, 60+}                 | protected       |
| `Target` | binary {0, 1} (e.g. loan approval)                           | outcome tested  |

The toy data are crafted so that global **demographic parity** holds  
(each race × age bucket has the same mean `Target`) **but young Blue people are
severely under-served**.  
MSD should discover exactly that subgroup.

<img src="../images/motivation_MSD.png" width="600">

In [1]:
dataset_path = "../data/01_data.csv"

target = "Target"
protected_list = ["Race", "Age"]

In [ ]:
from humancompatible.detect import compute_bias_csv

msd_val, rule_idx = compute_bias_csv(
    csv_path=dataset_path,
    target=target,
    protected_list=protected_list,
    method="MSD",
)

[INFO] Set parameter Username
[INFO] Set parameter LicenseID to value 2649381
[INFO] Academic license - for non-commercial use only - expires 2026-04-09


In [3]:
print(f"MSD value: {msd_val:.3f}")
print(f"Rule: {rule_idx}")

MSD value: 0.111
Rule: [np.int64(1), np.int64(3)]


## Interpreting the rule

`rule_idx` refers to columns of the **binarised** feature matrix.
You can rebuild the same binariser and map each index back to a literal:

In [4]:
import pandas as pd
from humancompatible.detect.data_handler import DataHandler
from humancompatible.detect.binarizer import Binarizer

df = pd.read_csv(dataset_path)
X = df[:].drop(columns=[target])
y = df[:][target]

categ_map = {col: X[col].unique() for col in X.columns}
dhandler = DataHandler.from_data(X, y, categ_map=categ_map)

binz = Binarizer(dhandler, target_positive_vals=[True])
encodings = binz.get_bin_encodings(include_binary_negations=True)

readable = " AND ".join(str(encodings[i]) for i in rule_idx)
print(readable)

Race = Blue AND Age = 0-18


## Conclusion

**MSD score**  
  The computed MSD of **0.111…** tells us that the subgroup  
  **`Race = Blue AND Age = 0–18`**  
  appears **11.1 percentage points more often** among the negative outcomes than among the positive outcomes.

**What this means**  
  In this toy dataset, "Blue youngsters" are the most under-served subgroup.  